In [1]:
import os
import sys

curr_dir = os.getcwd()
pkg_dir = os.path.join(os.path.dirname(curr_dir), "steam_sales", "steam_etl")
sys.path.append(pkg_dir)

In [2]:
import warnings

import pandas as pd
from db import get_db
from settings import Path
from sqlalchemy import text
from utils import print_steam_links

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 100)

### Data Extraction from SQL Database

In [3]:
def fetch_data(source: str):
    """
    Fetches data from a specified source and returns it as a pandas DataFrame.

    Parameters:
    source (str): The name of the source file containing the SQL query.

    Returns:
    pandas.DataFrame: The fetched data as a DataFrame.

    """
    with open(os.path.join(Path.sql_queries, source), "r") as f:
        query = text(f.read())

    with get_db() as db:
        result = db.execute(query)

    data = result.fetchall()
    columns = result.keys()
    df = pd.DataFrame(data, columns=columns)

    return df


steamspy_data = fetch_data("get_all_steamspy_data.sql")
steamspy_data.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,languages,genre,tags
0,10,Counter-Strike,Valve,Valve,,232593,6098,0.0,"10,000,000 .. 20,000,000",16697,55,219,22,999.0,999.0,0,13366,"English, French, German, Italian, Spanish - Sp...",Action,"{""FPS"": 4902, ""PvP"": 910, ""1980s"": 279, ""1990'..."
1,20,Team Fortress Classic,Valve,Valve,,7181,1089,0.0,"5,000,000 .. 10,000,000",1078,0,16,0,74.0,499.0,85,97,"English, French, German, Italian, Spanish - Sp...",Action,"{""FPS"": 327, ""Mod"": 39, ""Co-op"": 98, ""Funny"": ..."
2,30,Day of Defeat,Valve,Valve,,6166,671,0.0,"5,000,000 .. 10,000,000",187,0,41,0,74.0,499.0,85,81,"English, French, German, Italian, Spanish - Spain",Action,"{""FPS"": 798, ""War"": 158, ""Co-op"": 36, ""Retro"":..."
3,40,Deathmatch Classic,Valve,Valve,,2479,522,0.0,"5,000,000 .. 10,000,000",66,0,11,0,74.0,499.0,85,6,"English, French, German, Italian, Spanish - Sp...",Action,"{""FPS"": 150, ""Gore"": 20, ""Co-op"": 16, ""Retro"":..."
4,50,Half-Life: Opposing Force,Gearbox Software,Valve,,21457,1098,0.0,"2,000,000 .. 5,000,000",431,0,122,0,74.0,499.0,85,161,"English, French, German, Korean",Action,"{""FPS"": 919, ""Gore"": 58, ""Co-op"": 43, ""Retro"":..."


Creating a copy of `steam_data` dataset before starting the cleaning process.

In [4]:
raw_steamspy_data = steamspy_data.copy()

### Process Null values

Since the data is queried from SQL, some null vales are read as strings. 

In [5]:
def process_null(df):
    """
    Process null values in a DataFrame by replacing specific values with None.

    Args:
        df (pandas.DataFrame): The DataFrame to process.

    Returns:
        pandas.DataFrame: The processed DataFrame with null values replaced.

    """
    df = df.copy()

    convert_to_none = ["", "none", "null", "N/a", "N/A", "NA", "None", "n/a"]
    df.replace(convert_to_none, None, inplace=True)

    return df


raw_steam_data = process_null(raw_steamspy_data)
raw_steam_data.isnull().sum()

appid                  0
name                  17
developer            320
publisher            315
score_rank         74229
positive               0
negative               0
userscore              0
owners                 0
average_forever        0
average_2weeks         0
median_forever         0
median_2weeks          0
price                 29
initialprice          22
discount              22
ccu                    0
languages             94
genre                361
tags                 320
dtype: int64

### Dropping Columns and Rows 

Some columns are not needed as these are in `steam_data`. Others are temporary and change on a weekly/daily basis.

In [6]:
def process_col_rows(df):
    """
    Process the given DataFrame by dropping unnecessary columns and rows.

    Args:
        df (pandas.DataFrame): The DataFrame to be processed.

    Returns:
        pandas.DataFrame: The processed DataFrame.
    """
    df = df.copy()
    col_to_drop = [
        "score_rank",  # too many missing values
        "userscore",  # too little variance (most have 0)
        "genre",
        "developer",
        "publisher",
        "price",
        "initialprice",
        "discount",  # provided by Steam data
        "average_2weeks",
        "median_2weeks",
        "ccu",  # not interested in temporally specific columns
    ]

    # Drop missing games
    df = df.dropna(subset=["name"])
    df = df.drop(col_to_drop, axis=1)

    return df


col_row_df = process_col_rows(raw_steam_data)
col_row_df.head()

,appid,name,positive,negative,owners,languages,tags
0,10,Counter-Strike,231805,6061,"10,000,000 .. 20,000,000","English, French, German, Italian, Spanish - Sp...","{""FPS"": 4899, ""PvP"": 907, ""1980s"": 278, ""1990'..."
1,20,Team Fortress Classic,7136,1087,"50,000 .. 100,000","English, French, German, Italian, Spanish - Sp...","{""FPS"": 327, ""Mod"": 39, ""Co-op"": 98, ""Funny"": ..."
2,30,Day of Defeat,6140,668,"5,000,000 .. 10,000,000","English, French, German, Italian, Spanish - Spain","{""FPS"": 798, ""War"": 158, ""Co-op"": 36, ""Retro"":..."
3,40,Deathmatch Classic,2457,518,"100,000 .. 200,000","English, French, German, Italian, Spanish - Sp...","{""FPS"": 150, ""Gore"": 20, ""Co-op"": 16, ""Retro"":..."
4,50,Half-Life: Opposing Force,21262,1086,"2,000,000 .. 5,000,000","English, French, German, Korean","{""FPS"": 917, ""Gore"": 57, ""Co-op"": 43, ""Retro"":..."


### Processing Owners

In [7]:
def process_owners(df):
    """
    Process the 'owners' column in the given DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing the 'owners' column.

    Returns:
    pandas.DataFrame: The DataFrame with the 'owners' column processed.

    """
    df = df.copy()
    df["owners"] = (
        df["owners"]
        .apply(lambda x: tuple(map(lambda x: int(x) / 1000000, x.replace(",", "").split(" .. "))))
        .apply(lambda x: f"{x[0]} - {x[1]}")
    )

    return df


own_df = process_owners(col_row_df)
own_df.head()

,appid,name,positive,negative,owners,languages,tags
0,10,Counter-Strike,231805,6061,10.0 - 20.0,"English, French, German, Italian, Spanish - Sp...","{""FPS"": 4899, ""PvP"": 907, ""1980s"": 278, ""1990'..."
1,20,Team Fortress Classic,7136,1087,0.05 - 0.1,"English, French, German, Italian, Spanish - Sp...","{""FPS"": 327, ""Mod"": 39, ""Co-op"": 98, ""Funny"": ..."
2,30,Day of Defeat,6140,668,5.0 - 10.0,"English, French, German, Italian, Spanish - Spain","{""FPS"": 798, ""War"": 158, ""Co-op"": 36, ""Retro"":..."
3,40,Deathmatch Classic,2457,518,0.1 - 0.2,"English, French, German, Italian, Spanish - Sp...","{""FPS"": 150, ""Gore"": 20, ""Co-op"": 16, ""Retro"":..."
4,50,Half-Life: Opposing Force,21262,1086,2.0 - 5.0,"English, French, German, Korean","{""FPS"": 917, ""Gore"": 57, ""Co-op"": 43, ""Retro"":..."


### Processing Tags and Languages

In [8]:
print_steam_links(own_df[own_df["tags"].isnull()].sample(5, random_state=0))

Soft cookie: http://store.steampowered.com/app/1019070
Cursed Armor: http://store.steampowered.com/app/907090
Blood Bowl 2 - Review: http://store.steampowered.com/app/29017
Cadence: http://store.steampowered.com/app/362800
The Studio: http://store.steampowered.com/app/780150


ost of these games suggests that tags data wasn't supplied.

In [9]:
print_steam_links(own_df[own_df["languages"].isnull()].sample(5, random_state=0))

Puzzlegeddon: http://store.steampowered.com/app/8740
Endless Space VIP: http://store.steampowered.com/app/212090
Arcadia - Beta: http://store.steampowered.com/app/72520
HOMEFRONT Demo: http://store.steampowered.com/app/55130
Altitude - Demo: http://store.steampowered.com/app/41310


On closer inspection, games having missing languages are demos, beta or do not exist.

In [10]:
def process_tag_lang(df):
    """
    Process the dataframe by dropping rows with missing values in the 'languages' and 'tags' columns.

    Args:
        df (pandas.DataFrame): The input dataframe.

    Returns:
        pandas.DataFrame: The processed dataframe with missing values dropped.

    """
    df = df.copy()
    df = df.dropna(subset=["languages", "tags"])

    return df


tag_lang_df = process_tag_lang(own_df)

### Final Steps

All columns are devoid of null values.

In [11]:
tag_lang_df.isnull().sum()

appid        0
name         0
positive     0
negative     0
owners       0
languages    0
tags         0
dtype: int64

Renaming columns.

In [12]:
def rename(df):
    """
    Renames the columns of the given DataFrame.

    Args:
        df (pandas.DataFrame): The DataFrame to be renamed.

    Returns:
        pandas.DataFrame: The renamed DataFrame.
    """
    df = df.copy()
    df = df.rename(
        {
            "tags": "steamspy_tags",
            "positive": "positive_ratings",
            "negative": "negative_ratings",
            "owners": "owners_in_millions",
        },
        axis=1,
    )

    return df


clean_steamspy_data = rename(tag_lang_df)

Memory usage analysis of raw data and clean data.

In [13]:
raw_steamspy_data.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74273 entries, 0 to 74272
Columns: 20 entries, appid to tags
dtypes: float64(3), int64(8), object(9)
memory usage: 63.1 MB


In [18]:
clean_steamspy_data.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 73925 entries, 0 to 74272
Columns: 7 entries, appid to steamspy_tags
dtypes: int64(3), object(4)
memory usage: 37.6 MB
